In [ ]:
# !pip install import-ipynb

#access drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial

x_path= "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/clean_data/ohc3.csv"
y_path= "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/clean_data/depVar"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial


In [ ]:
%run Splitting_data.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial # Change path to yours'
/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial


In [ ]:
X_train, X_val, y_train, y_val = training_set(x_path, y_path)

In [4]:
import xgboost
from matplotlib import pyplot as plt

eval_set=[(X_train, y_train), (X_val, y_val)]

xgb_reg = xgboost.XGBRegressor(n_estimators=3000,
                               max_depth=12,
                               subsample=1,
                               max_bin=500, # Maximum number of discrete bins to bucket continuous features. Increasing this number improves the optimality of splits at the cost of higher computation time.
                               objective="reg:squarederror", # for iterative learning
                               eval_metric="rmse", # for early-stopping evaluation using eval_set
                               early_stopping_rounds=10,
                               max_delta_step=0, # Maximum delta step we allow each leaf output to be. If the value is set to 0, it means there is no constraint. If it is set to a positive value, it can help making the update step more conservative. Usually this parameter is not needed, but it might help in logistic regression when class is extremely imbalanced. Set it to value of 1-10 might help control the update.
                               verbosity=1,
                               n_jobs=-1,
                              )
# see more parameters in https://xgboost.readthedocs.io/en/stable/parameter.html#parameters-for-tree-booster

xgb_reg.fit(X_train, y_train, 
            eval_set= eval_set, 
            #verbose=False
           )

results = xgb_reg.evals_result()
lowest_score = round(min(results["validation_1"]["rmse"]), 2)

print(f"Optimal number of trees: {xgb_reg.best_ntree_limit}\n RMSE: {lowest_score}")

# plot graph
plt.figure(figsize=(10,7))
plt.plot(results["validation_0"]["rmse"], label="Training loss")
plt.plot(results["validation_1"]["rmse"], label="Validation loss")
plt.axvline(xgb_reg.best_ntree_limit, color="gray", label="Optimal tree number")
plt.xlabel("Number of trees")
plt.ylabel("Loss")
plt.legend()

TypeError: ignored

Important parameters: 


*   early_stopping: tells the model after how many rounds to stop after it finds the smallest number of the evaluation metric
*   eval_metric: evaluation metric must be specified 

